In [1]:

from langchain.chains.combine_documents import collapse_docs, split_list_of_docs
from langchain_community.chat_models import ChatOpenAI

# from langchain.chat_models import ChatAnthropic
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.schema import Document, StrOutputParser
from langchain_core.prompts import format_document
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from functools import partial


/home/dp/hdd/gits/EncyclopediaChat


In [2]:
from ragchat.html_cleaner import HtmlCleaner
from ragchat.html_patterns import HtmlPatterns


In [3]:
cleaner = HtmlCleaner(parser="inscriptis", debug=True, max_pages=None)
clean_text_dict = cleaner.get_clean_text_dict()


In [4]:
l_lens = []
c_lens = []
for fname in list(clean_text_dict.keys()):
    cleaned = clean_text_dict[fname]["cleaned"]
    c_lens.append((len(cleaned), fname))
    l_lens.append((len(cleaned.splitlines()), fname))

c_lens.sort()
l_lens.sort()


In [5]:
idx = -1
pg = c_lens[idx][1]
text = clean_text_dict[pg]["cleaned"]
title = clean_text_dict[pg]["title"]
all_text = clean_text_dict[pg]["all_text"]


In [6]:
print(title, text)


Security Advisories TRUST CENTER
         Overview
         Security
         Incident Response
         Privacy
         Compliance
         Availability
         Advisories
         Contact Us

        AUTODESK TRUST CENTER

        Timely and transparent notice

        Autodesk publishes important information regarding security vulnerabilities that could affect specific versions of Autodesk products or services.

          REPORT AN ISSUE

         Latest security bulletins and advisories

         This page contains important information regarding security vulnerabilities that could affect specific versions of Autodesk products or services.

              Important security update

              Autodesk recently determined that an unauthorized third-party obtained access to portions of internal systems. Our findings show that sensitive data about our customers and their projects or products have not been compromised. We immediately took steps to contain the incident. Forensic ana

In [7]:
doc = Document(
    page_content=text,
    metadata={"page": pg, "title": title},
)


In [8]:
document_prompt = PromptTemplate.from_template("title: {title} \n\n\n{page_content}")
partial_format_document = partial(format_document, prompt=document_prompt)


In [9]:
llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0)


/home/dp/hdd/gits/EncyclopediaChat/env/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(


In [10]:
llm.get_num_tokens(text)


7188

In [11]:
template = f"""Answer the question based only on the following context:
    {text}
    
    Question: {'{question}'}
    """
prompt = ChatPromptTemplate.from_template(template)
query_chain = {"question": RunnablePassthrough()} | prompt | llm | StrOutputParser()


In [12]:
# query_chain = (
#     {"context": doc,
#      "question": RunnablePassthrough()}
#     | PromptTemplate.from_template(
#         )
#     | llm
#     | StrOutputParser()
# )


In [13]:
reply = query_chain.invoke(
    "What was the most recent security incident for AutoCad Desktop Software?"
)
reply


'The most recent security incident for AutoCad Desktop Software was the revocation of the Autodesk code signing certificate on August 5th for all software code signed after July 10, 2022 (00:00 GMT). This incident was identified under the Autodesk ID: ADSK-SA-2023-0021 on 11/15/2023, Thursday.'

In [14]:
all_lines = all_text.splitlines()
srch = [l_i for l_i, l in enumerate(all_lines) if "ADSK-SA-2023-0021" in l]
for l_i in srch:
    print("\n".join(all_lines[l_i - 5 : l_i + 5]))



                                                * Revocation of Autodesk code signing certificate in the Autodesk AutoCAD Desktop Software

                                                  Autodesk has revoked the certificate on August 5th for all software code signed after July 10, 2022 (00:00 GMT). Autodesk has issued updates signed using a new digital certificate for AutoCAD-based products. Please note, only the 2024 versions of software are affected by this new digital certificate.

                                                  Autodesk ID: ADSK-SA-2023-0021
                                                  11/15/2023, Thursday

                                                  Learn more


                                                * Revocation of Autodesk code signing certificate in the Autodesk AutoCAD Desktop Software

                                                  Autodesk has revoked the certificate on August 5th for all software code signed after July 10, 2022 

In [15]:
pg


'adsk-c452733c1c8246628e34a73313109bef.html'